## Task 2:
There is a large body of literature surrounding racial bias within the criminal legal system. Particularly, there is a robust literature related to the presence of racial bias (or lack thereof) when examining police stops and behavior. Within this task, you will be exploring whether racial bias exists within police **vehicle** search behavior in Philadelphia for 2021. Particularly, you will be assessing the impact race has on an individual's likelihood to be searched. 

## Set directories and import required packages

In [235]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pathlib
import ephem
import pytz

In [236]:
# Set directory to the parent directory of the current working directory (repo directory)
repo_dir = pathlib.Path.cwd().parent

# Set pandas display options to show all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Load data

In [237]:
# PPD stops data for 2021.
df_stops = pd.read_csv(r"https://phl.carto.com/api/v2/sql?filename=car_ped_stops&format=csv&skipfields=cartodb_id,the_geom,the_geom_webmercator&q=SELECT%20*,%20ST_Y(the_geom)%20AS%20lat,%20ST_X(the_geom)%20AS%20lng%20FROM%20car_ped_stops%20WHERE%20datetimeoccur%20%3E=%20%272021-01-01%27")

In [238]:
df_stops['datetimeoccur'].dtype

dtype('O')

In [239]:
# Define datetimeoccur as EST
df_stops['datetimeoccur'] = pd.to_datetime(df_stops['datetimeoccur']).dt.tz_convert(None).dt.tz_localize('EST')

## Explore Data

In [240]:
df_stops.shape

(267814, 24)

In [241]:
df_stops.head()

,objectid,id,datetimeoccur,weekday,location,districtoccur,psa,stopcode,stoptype,inside_or_outside,gender,race,age,individual_frisked,individual_searched,individual_arrested,individual_contraband,vehicle_frisked,vehicle_searched,vehicle_contraband,point_x,point_y,lat,lng
0,2840052,40855655,2022-08-23 02:45:00-05:00,MONDAY,3300 BLOCK CHESTERFIELD RD,8,2,2702,vehicle,Outside,Male,White - Non-Latino,21.0,0,0,0,0,0,0,0,-74.995892,40.075055,40.075055,-74.995892
1,2840053,40857764,2022-08-27 22:11:00-05:00,SATURDAY,3300 BLOCK N 16TH ST,39,2,2702,vehicle,Outside,Male,Black - Non-Latino,64.0,0,0,0,0,0,0,0,-75.155806,40.003709,40.003709,-75.155806
2,2840054,40857245,2022-08-26 17:19:00-05:00,FRIDAY,3300 BLOCK N Rorer,25,3,2701,pedestrian,Outside,Male,White - Non-Latino,65.0,1,1,1,0,0,0,0,-75.119868,39.999198,39.999198,-75.119868
3,2840055,40860055,2022-09-01 23:20:00-05:00,THURSDAY,3500 BLOCK HARTEL,15,3,2702,vehicle,Outside,Male,American Indian,52.0,0,0,0,0,0,0,0,-75.032678,40.042088,40.042088,-75.032678
4,2840056,40857105,2022-08-26 00:54:00-05:00,THURSDAY,3500 BLOCK KENSINGTON AVE,24,1,2702,vehicle,Outside,Male,White - Non-Latino,18.0,1,0,0,0,1,0,0,-75.105256,40.000890,40.000890,-75.105256


In [242]:
# Check for duplicates by objectid and id
print("Duplicates by objectid: ", df_stops.duplicated(subset=['objectid']).sum())
print("Duplicates by id: ", df_stops.duplicated(subset=['id']).sum())

Duplicates by objectid:  0
Duplicates by id:  0


In [243]:
# Create age group column
df_stops['age_group'] = pd.cut(df_stops['age'], bins=[0, 18, 25, 35, 45, 55, 65, df_stops['age'].max()], labels=['0-18', '19-25', '26-35', '36-45', '46-55', '56-64', '65+'])

# Create month and year indicator columns using datetimeoccur column
df_stops['datetimeoccur'] = pd.to_datetime(df_stops['datetimeoccur'])
df_stops['month'] = df_stops['datetimeoccur'].dt.month
df_stops['year'] = df_stops['datetimeoccur'].dt.year

In [244]:
print("Number of unique districts:")
print(df_stops['districtoccur'].nunique())

print("Number of unique psas:")
print(df_stops['psa'].nunique())

Number of unique districts:
22
Number of unique psas:
5


In [245]:
cols_to_investigate = ['race', 'gender', 'stoptype', 'inside_or_outside', 
                        'age_group', 'individual_frisked',	'individual_searched',	
                        'individual_arrested',	'individual_contraband',
                        'vehicle_frisked',	'vehicle_searched',	'vehicle_contraband',
                        'month', 'year', 'stopcode']

for col in cols_to_investigate:
    print(f"Distribution of {col}:")
    print(df_stops[col].value_counts(normalize=True, dropna=False))
    print()

Distribution of race:
Black - Non-Latino    0.713462
White - Non-Latino    0.144085
White - Latino        0.086971
Asian                 0.020074
Unknown               0.018487
Black - Latino        0.013397
American Indian       0.003525
Name: race, dtype: float64

Distribution of gender:
Male      0.732131
Female    0.267634
NaN       0.000235
Name: gender, dtype: float64

Distribution of stoptype:
vehicle       0.913724
pedestrian    0.086276
Name: stoptype, dtype: float64

Distribution of inside_or_outside:
Outside    0.994287
Inside     0.005713
Name: inside_or_outside, dtype: float64

Distribution of age_group:
26-35    0.334210
19-25    0.265438
36-45    0.170600
46-55    0.098755
56-64    0.059534
0-18     0.048717
65+      0.021328
NaN      0.001419
Name: age_group, dtype: float64

Distribution of individual_frisked:
0    0.920322
1    0.079678
Name: individual_frisked, dtype: float64

Distribution of individual_searched:
0    0.931329
1    0.068671
Name: individual_searched, 

In [246]:
# Prompt asked for 2021 data only so I will filter the data to 2021
df_stops = df_stops[df_stops['year'] == 2021]

# Prompt asked for only vehicle stops so I will filter the data to vehicle stops
df_stops = df_stops[df_stops['stoptype'] == 'vehicle']

In [247]:
df_stops.shape

(125041, 27)

## Create model to look into stop behavior

In [232]:
df_stops.head()

,objectid,id,datetimeoccur,weekday,location,districtoccur,psa,stopcode,stoptype,inside_or_outside,gender,race,age,individual_frisked,individual_searched,individual_arrested,individual_contraband,vehicle_frisked,vehicle_searched,vehicle_contraband,point_x,point_y,lat,lng,age_group,month,year,search_or_frisk,hit,light_out,hour,search_or_frisk_all,search_or_frisk_vehicle,hit_vehicle
11927,2527272,4154741,2021-01-01 01:18:00-05:00,THURSDAY,6600 BLOCK BUIST AVE,12,1,2702,vehicle,Outside,Male,Black - Non-Latino,22.0,0,0,0,0,0,0,0,NaN,NaN,39.920123,-75.230176,19-25,1,2021,0,0,0,1,0,0.0,NaN
11928,2535914,4154687,2021-01-01 00:00:00-05:00,THURSDAY,5800 BLOCK Chester Ave,12,2,2702,vehicle,Outside,Male,Black - Non-Latino,22.0,0,0,0,0,0,0,0,NaN,NaN,39.934524,-75.228858,19-25,1,2021,0,0,0,0,0,0.0,NaN
11929,2534799,4154855,2021-01-01 00:35:00-05:00,THURSDAY,5700 BLOCK ogontz,35,3,2702,vehicle,Outside,Male,Black - Non-Latino,40.0,0,0,0,0,0,0,0,NaN,NaN,40.041715,-75.150782,36-45,1,2021,0,0,0,0,0,0.0,NaN
11930,2536079,4154844,2021-01-01 04:15:00-05:00,THURSDAY,1300 BLOCK lindley,35,2,2702,vehicle,Outside,Male,Black - Non-Latino,34.0,0,0,0,0,0,0,0,NaN,NaN,40.029727,-75.144733,26-35,1,2021,0,0,0,4,0,0.0,NaN
11931,2536080,4154845,2021-01-01 04:15:00-05:00,THURSDAY,1300 BLOCK lindley,35,2,2702,vehicle,Outside,Female,Black - Non-Latino,30.0,0,0,0,0,0,0,0,NaN,NaN,40.029727,-75.144733,26-35,1,2021,0,0,0,4,0,0.0,NaN


In [249]:
# Create search_or_frisk column
df_stops['search_or_frisk_all'] = np.where((df_stops['individual_frisked'] == 1) | (df_stops['individual_searched'] == 1) | (df_stops['vehicle_frisked'] == 1) | (df_stops['vehicle_searched'] == 1), 1, 0)
df_stops['search_or_frisk_vehicle'] = np.where((df_stops['vehicle_frisked'] == 1) | (df_stops['vehicle_searched'] == 1), 1, 
                                               np.where((df_stops['vehicle_frisked'] == 0) | (df_stops['vehicle_searched'] == 0), 0, np.nan))

In [252]:
df_stops['search_or_frisk_all'].value_counts(normalize=True, dropna=False)

0    0.865772
1    0.134228
Name: search_or_frisk_all, dtype: float64

In [253]:
fx = 'search_or_frisk_all ~ C(race, Treatment(reference=\'White - Non-Latino\')) + C(gender) + C(weekday) + C(month) + C(year) + C(districtoccur) + C(stoptype) + C(age_group) '
model = smf.ols(formula=fx, data=df_stops)
res = model.fit(cov_type='HC3')
print(res.summary())

                             OLS Regression Results                            
Dep. Variable:     search_or_frisk_all   R-squared:                       0.094
Model:                             OLS   Adj. R-squared:                  0.094
Method:                  Least Squares   F-statistic:                     269.2
Date:                 Sun, 12 Feb 2023   Prob (F-statistic):               0.00
Time:                         08:21:16   Log-Likelihood:                -36613.
No. Observations:               124783   AIC:                         7.333e+04
Df Residuals:                   124731   BIC:                         7.384e+04
Df Model:                           51                                         
Covariance Type:                   HC3                                         
                                                                               coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------

In [254]:
# Print summary to latex
print(res.summary().as_latex())
# Save summary to txt file
outfile = repo_dir.joinpath('output', 'search_or_frisk_task2.tex')
with open(outfile, 'w') as f:
    f.write(res.summary().as_latex())

\begin{center}
\begin{tabular}{lclc}
\toprule
\textbf{Dep. Variable:}                                                           & search\_or\_frisk\_all & \textbf{  R-squared:         } &     0.094   \\
\textbf{Model:}                                                                   &          OLS           & \textbf{  Adj. R-squared:    } &     0.094   \\
\textbf{Method:}                                                                  &     Least Squares      & \textbf{  F-statistic:       } &     269.2   \\
\textbf{Date:}                                                                    &    Sun, 12 Feb 2023    & \textbf{  Prob (F-statistic):} &     0.00    \\
\textbf{Time:}                                                                    &        08:21:30        & \textbf{  Log-Likelihood:    } &   -36613.   \\
\textbf{No. Observations:}                                                        &         124783         & \textbf{  AIC:               } & 7.333e+04   \\
\textbf{Df R

## Create model to look into hit rate


In [259]:
# Create a hit rate column
df_stops['hit'] = np.where((df_stops['individual_contraband'] == 1) & ((df_stops['individual_searched'] == 1) | (df_stops['individual_frisked']==1)), 1,
                                np.where((df_stops['vehicle_contraband'] == 1) & ((df_stops['vehicle_searched'] == 1) | (df_stops['vehicle_frisked'] == 1)), 1,
                                np.where((df_stops['vehicle_contraband'] == 0) & ((df_stops['vehicle_searched'] == 1) | (df_stops['vehicle_frisked'] == 1)), 0,
                                np.where((df_stops['individual_contraband'] == 0) & ((df_stops['vehicle_searched'] == 1) | (df_stops['vehicle_frisked'] == 1)), 0, np.nan))))
print("Hit rate overall:")
print(df_stops['hit'].value_counts(normalize=True, dropna=False))

# Create a hit rate column
df_stops['hit_vehicle'] = np.where((df_stops['vehicle_contraband'] == 1) & (df_stops['search_or_frisk_vehicle']==1), 1, 
                                   np.where((df_stops['vehicle_contraband'] == 0) & (df_stops['search_or_frisk_vehicle']==1), 0, np.nan))
print("Hit rate for vehicle stops:")
df_stops['hit_vehicle'].value_counts(normalize=True, dropna=False)

Hit rate overall:
NaN    0.888740
0.0    0.086572
1.0    0.024688
Name: hit, dtype: float64
Hit rate for vehicle stops:


NaN    0.892635
0.0    0.089275
1.0    0.018090
Name: hit_vehicle, dtype: float64

In [261]:
# Drop districtoccur with less than 100 stops (to ensure sufficient sample size within each district)
df_stops = df_stops.groupby('districtoccur').filter(lambda x: len(x) > 100)

In [275]:
fx = 'hit ~ C(race, Treatment(reference=\'White - Non-Latino\')) + C(gender) + C(weekday) + C(month) + C(districtoccur) + C(age_group) '
model = smf.ols(formula=fx, data=df_stops)
res = model.fit(cov_type='HC3')
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                    hit   R-squared:                       0.043
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     10.05
Date:                Sun, 12 Feb 2023   Prob (F-statistic):           1.90e-74
Time:                        09:04:09   Log-Likelihood:                -7196.9
No. Observations:               13884   AIC:                         1.450e+04
Df Residuals:                   13833   BIC:                         1.488e+04
Df Model:                          50                                         
Covariance Type:                  HC3                                         
                                                                               coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------

In [263]:
# Print summary to latex
print(res.summary().as_latex())
# Save summary to txt file
outfile = repo_dir.joinpath('output', 'hit_rate_task2.tex')
with open(outfile, 'w') as f:
    f.write(res.summary().as_latex())

\begin{center}
\begin{tabular}{lclc}
\toprule
\textbf{Dep. Variable:}                                                           &       hit        & \textbf{  R-squared:         } &     0.043   \\
\textbf{Model:}                                                                   &       OLS        & \textbf{  Adj. R-squared:    } &     0.040   \\
\textbf{Method:}                                                                  &  Least Squares   & \textbf{  F-statistic:       } &     10.05   \\
\textbf{Date:}                                                                    & Sun, 12 Feb 2023 & \textbf{  Prob (F-statistic):} &  1.90e-74   \\
\textbf{Time:}                                                                    &     08:26:36     & \textbf{  Log-Likelihood:    } &   -7196.9   \\
\textbf{No. Observations:}                                                        &       13884      & \textbf{  AIC:               } & 1.450e+04   \\
\textbf{Df Residuals:}                          

### Define function to calculate if it is light out

In [265]:
def light_out(time):
    # Create a city object for Philadelphia
    phila = ephem.Observer()
    phila.lon = '-75.1652'
    phila.lat = '39.9526'

    # convert time to UTC
    # time = time.tz_convert('UTC')
    
    # Set the city object's time to the timestamp
    phila.date = time

    # Get the sun object
    sun = ephem.Sun()
    # Compute the sun's position
    sun.compute(phila)
    # Get the sun's altitude
    altitude = sun.alt
    # Define twilight
    twilight = -12 * ephem.degree

    # Return 1 if sun is below twilight, 0 otherwise

    return 1 if altitude > twilight else 0

In [266]:
# Create a column for whether it was light out or not using the light_out function
df_stops['light_out'] = df_stops['datetimeoccur'].apply(light_out)

In [267]:
# Get the hour of day from the datetimeoccur column
df_stops['hour'] = df_stops['datetimeoccur'].dt.hour

# Look at distribution of hour by light_out
print("Hour distribution by light_out==1:")
print(df_stops[df_stops['light_out'] == 1]['hour'].value_counts())
print()
print("Hour distribution by light_out==0:")
print(df_stops[df_stops['light_out'] == 0]['hour'].value_counts())

Hour distribution by light_out==1:
15    5095
14    4631
16    4626
17    4080
13    3366
18    2292
6     1712
12    1616
5     1569
4     1504
19    1210
7     1077
20     816
3      611
8      601
11     534
9      350
10     292
Name: hour, dtype: int64

Hour distribution by light_out==0:
0     15374
23    14897
1     13315
22    11237
2     10270
21     8322
3      5574
20     3900
4      2380
19     1596
5      1049
18      934
6       104
17       99
Name: hour, dtype: int64


## Run models when light out and when not

In [277]:
# Subset the data to light_out==1  and light_out==0 and hours that exist in both dataframes
df_light_out = df_stops[df_stops['light_out'] == 1]
df_dark_out = df_stops[df_stops['light_out'] == 0]
# Keep three hours with most overlap between light_out==1 and light_out==0
hours = [18, 19, 20]
df_light_out = df_light_out[df_light_out['hour'].isin(hours)]
df_dark_out = df_dark_out[df_dark_out['hour'].isin(hours)]

In [279]:
fx = 'search_or_frisk_all ~ C(race, Treatment(reference=\'White - Non-Latino\'))*C(light_out) + C(gender) + C(month) + C(weekday) + C(districtoccur) + C(age_group) '
model = smf.ols(formula=fx, data=df_stops)
res = model.fit(cov_type='HC3')
print(res.summary())

                             OLS Regression Results                            
Dep. Variable:     search_or_frisk_all   R-squared:                       0.094
Model:                             OLS   Adj. R-squared:                  0.094
Method:                  Least Squares   F-statistic:                     241.0
Date:                 Sun, 12 Feb 2023   Prob (F-statistic):               0.00
Time:                         09:13:06   Log-Likelihood:                -36599.
No. Observations:               124776   AIC:                         7.331e+04
Df Residuals:                   124718   BIC:                         7.388e+04
Df Model:                           57                                         
Covariance Type:                   HC3                                         
                                                                                                 coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------

In [280]:
fx = 'hit ~ C(race, Treatment(reference=\'White - Non-Latino\'))*C(light_out) + C(gender) + C(weekday) + C(month) + C(districtoccur) + C(age_group) '
model = smf.ols(formula=fx, data=df_stops)
res = model.fit(cov_type='HC3')
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                    hit   R-squared:                       0.045
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     9.190
Date:                Sun, 12 Feb 2023   Prob (F-statistic):           5.27e-75
Time:                        09:13:15   Log-Likelihood:                -7186.5
No. Observations:               13884   AIC:                         1.449e+04
Df Residuals:                   13826   BIC:                         1.493e+04
Df Model:                          57                                         
Covariance Type:                  HC3                                         
                                                                                                 coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------